In [1]:
import cv2
import numpy as np
import os
from keras.applications.xception import Xception
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, Activation, ZeroPadding2D, Dense, MaxPooling2D, GlobalAveragePooling2D, Dropout, Input, merge, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model
import keras.optimizers as optimizers
import keras.applications.inception_resnet_v2 as inception_resnet_v2

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(rotation_range=270,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True
            )

In [3]:
train = datagen.flow_from_directory('/input/flowers2/flowers_train',
                                              target_size = (299, 299),
                                              batch_size = 16,
                                              class_mode = 'categorical',
                                              
                                              )
validation = datagen.flow_from_directory('/input/flowers2/flowers_test',
                                              target_size = (299, 299),
                                              batch_size = 16,
                                              class_mode = 'categorical'
                                              )

Found 2517 images belonging to 4 classes.
Found 250 images belonging to 4 classes.


In [4]:
base_model = Xception( weights='imagenet', include_top=True)

90415104/91884032 [============================>.] - ETA: 0s

In [37]:
#base_model.summary()
x = Dense(1024)(base_model.layers[-2].output)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)
new_model = Model(inputs=base_model.input, outputs= x)

In [38]:
for layer in base_model.layers[:]:
    layer.trainable = True

In [48]:
#optimizer = optimizers.Adam(lr=0.001)
optimizer=optimizers.SGD(lr=0.00001, decay=1e-6, momentum=0.9, nesterov=True)
new_model.compile(loss='categorical_crossentropy',
          optimizer=optimizer,
          metrics=['accuracy'])

In [49]:
new_model.fit_generator(train, steps_per_epoch = 158, validation_data=validation, validation_steps=16, epochs=1)

Epoch 1/1
158/158 [==============================] - 209s - loss: 0.2961 - acc: 0.9133 - val_loss: 0.0712 - val_acc: 0.9840


In [50]:
new_model.save_weights("flower_rec.h5")